## Phase 2: Data Quality Assessment

### Motivation
What’s broken, missing, or suspicious?

### Questions to answer
1. Are types correct for each variable?
2. Do values make domain sense?
3. Where is data missing, and how much?
4. Are there duplicates?
5. Are there outliers?
6. Can I trust this data as-is?



In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/amazon_sales_2025_INR.csv',
                 parse_dates=['Date'],
                 na_values=["NA", "", "null"])


In [ ]:
df.dtypes

**Interpretation**

Date is correctly parsed as datetime. 
Numeric fields: Quantity, Unit_Price_INR, Total_Sales_INR, Review_Rating 
Categorical fields Product_Category, State, Payment_Method, etc. they appear as object.

In [ ]:
df.isna().sum()

**Interpretation**
After verifying using `.isna().sum()`, no missing values were found across the dataset. Therefore, no further action on missing data is required.

In [ ]:
df.duplicated().sum()

**Interpretation**
  
There are no repeated rows.

In [ ]:
df['Quantity'].describe()
df[~df['Quantity'].between(1, 5)]

**Interpretation**

For Quantity variable, the range is between 1-5, which make sense and no values outside 1-5 this is good.

In [ ]:
df['Review_Rating'].unique()
df[~df['Review_Rating'].between(1, 5)]

**Interpretation**

Also, no values outside 1-5 this is good. And the number identification is understandable.

In [ ]:
df[df['Unit_Price_INR'] <= 0]
df[df['Total_Sales_INR'] <= 0]

**Interpretation**

All values fall into a realistic market range. Both prices and sales > 0. 

In [ ]:
df['Product_Category'].unique()

In [ ]:
df['Payment_Method'].unique()

In [ ]:
df['Delivery_Status'].unique()

In [ ]:
df['State'].nunique()

**Interpretation**
1. Product_Category, Payment_method and delivery_status have limited categories. They are good for grouping.
2. States come with number 28 means 28 Indian States, which is reasonable.
3. There are no strange category values.

In [ ]:
df['unitprice_z'] = (df['Unit_Price_INR'] - df['Unit_Price_INR'].mean()) / df['Unit_Price_INR'].std()
df['sales_z'] = (df['Total_Sales_INR'] - df['Total_Sales_INR'].mean()) / df['Total_Sales_INR'].std()

df[(df['unitprice_z'].abs() > 3) | (df['sales_z'].abs() > 3)].head()


**Explanation**

The dataset reviewed from z-score means it doesn't contain statistical outliers.

In [ ]:
Q1 = df['Total_Sales_INR'].quantile(0.25)
Q3 = df['Total_Sales_INR'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[(df['Total_Sales_INR'] < Q1 - 1.5*IQR) |
              (df['Total_Sales_INR'] > Q3 + 1.5*IQR)]
outliers.head()


In [ ]:
df['Total_Sales_INR'].plot(kind='box', figsize=(5,3))

**Explanation**

To identify outliers, I applied IQR method, especially for total_sales_INR.The filtered results show 6 transactions with significantly higher sales amounts. These outliers come from categories such as home & kitchen, clothing, books and electronics. There are no negative values appear and the date and customer ID variables look normal. 
Therefore, even though there are outliers, it doesn't mean there are data quality issues requiring removal for those transactions.     

In [ ]:
Q1 = df['Unit_Price_INR'].quantile(0.25)
Q3 = df['Unit_Price_INR'].quantile(0.75)
IQR = Q3 - Q1

outliers = df[(df['Unit_Price_INR'] < Q1 - 1.5*IQR) |
              (df['Unit_Price_INR'] > Q3 + 1.5*IQR)]
outliers.head()

**Explanation**

There are no outliers in Unit_Price_INR column.

## Conclusion:

Even though there are 0 missing values, but as the initial step in phase 1, there are no missing categories or incomplete price or quantity fields. Also all the customer id is unique. The data is well structured with no wrong dtypes. I can not find any impossible or invalid values so far and all values fall into realistic business range. The outliers appeared but it's real high-value transactions so it's reasonable.Furthermore, the dataset not have duplicates. 
Therefore, I assume the dataset can be trusted as-is for analysis.  

### Hypothesis Generation
1. For outliers in Total_Sales_INR, it might caused from premium items. Those numbers are reasonable because the Date and Customer ID fields show normally.
2. The dataset assume has no missing values based on recent check and missing values assume be cleaned by provider. 
3. All categorical and numerical variables fall within reasonable domains.
### Iteration Signals:
We may need to verify whether these extreme values are expected business behavior or not. Also, because the dataset contains no missing values, there is no need to revisit data loading parameters or investigate data loss. 
